In [ ]:
from collections.abc import Callable

import numpy as np

from guppylang import guppy
from guppylang.std.builtins import (
    array, 
    comptime, 
    result, 
    owned
)

from guppylang.std.angles import pi

from guppylang.std.quantum import (
    qubit, 
    x, 
    h, 
    cx, 
    rz, 
    measure, 
    measure_array, 
    toffoli,
)

n = guppy.nat_var("n")

In [ ]:
@guppy
def grover_search(
    x_array: array[int, n],
    q_array: array[qubit, n] @owned,
    phase_oracle: Callable[
        [array[int, n],
        array[qubit, n]],
        bool
    ],
    n_iterations: int
) -> array[bool, n]:
    for _ in range(n_iterations):
        ancilla_result = phase_oracle(x_array, q_array)
        operator(q_array)
    return measure_array(q_array)

grover_search.check()

Use 3-qubits as the control qubit and the fourth qubit as the target qubit.

In [ ]:
@guppy
def c3x(q: array[qubit, n], target: qubit) -> None:
    rz(q[0], pi * 0.125)
    rz(q[1], pi * 0.125)
    rz(q[2], pi * 0.125)
    h(target)
    cx(q[0],q[1])
    rz(target, pi * 0.125)
    rz(q[1], pi * 3.875)
    cx(q[0],q[1])
    cx(q[1],q[2])
    rz(q[2], pi * 3.875)
    cx(q[0],q[2])
    rz(q[2], pi * 0.125)
    cx(q[1],q[2])
    rz(q[2], pi * 3.875)
    cx(q[0],q[2])
    cx(q[2],target)
    rz(target, pi * 3.875)
    cx(q[1],target)
    rz(target, pi * 0.125)
    cx(q[2],target)
    rz(target, pi * 3.875)
    cx(q[0],target)
    rz(target, pi * 0.125)
    cx(q[2],target)
    rz(target, pi * 3.875)
    cx(q[1],target)
    rz(target, pi * 0.125)
    cx(q[2],target)
    rz(target, pi * 3.875)
    cx(q[0],target)
    h(target)

c3x.check()

In [ ]:
@guppy
def mark(
    x_array: array[int, n],
    q_array: array[qubit, n]
) -> None:
    for i in range(n):
        if not x_array[i]:
            x(q_array[i])

mark.check()

In [ ]:
@guppy
def oracle(
    x_array: array[int, n],
    q_array: array[qubit, n]
) -> bool:
    ancilla = qubit()
    x(ancilla)
    h(ancilla)
    mark(x_array, q_array)
    c3x(q_array, ancilla)
    mark(x_array, q_array)
    h(ancilla)
    x(ancilla)
    b = measure(ancilla)
    return b

oracle.check()

In [ ]:
@guppy
def operator(
    q_array: array[qubit, n]
) -> None:
    for i in range(n):
        h(q_array[i])
        x(q_array[i])
    
    h(q_array[2])
    toffoli(q_array[0], q_array[1], q_array[2])
    h(q_array[2])

    for i in range(n):
        x(q_array[i])
        h(q_array[i])

operator.check()

In [ ]:
def approximate(
    N: int
) -> int:
    return np.ceil(0.25 * np.pi / np.arcsin(1/np.sqrt(2**N)) - 0.5)

In [ ]:
marked_state = [1, 1, 1]
N = len(marked_state)
K = int(approximate(N))
print(f"Marked State: {''.join(map(str, marked_state))}")
print(f"N: {N}")
print(f"K: {K}")

@guppy
def main() -> None:
    q_array = array(qubit() for _ in range(comptime(N)))
    for i in range(comptime(N)):
        h(q_array[i])
    x_array = array(i for i in comptime(marked_state))
    data_result = grover_search(x_array, q_array, oracle, comptime(K))
    result("data", data_result)

hugr_program = main.compile()